In [11]:
%matplotlib inline
from query.scripts.script_util import *
from pprint import pprint

In [15]:
print(Frame.objects.count()/15)

1341257


In [19]:
# Get total number of detected female vs male faces
gender_frames = Gender.objects.annotate(count=Count('tvnews_faceinstance__frame')).values('name', 'count')
total = gender_frames[0]['count'] + gender_frames[1]['count']
print('Female: {:.1f}%, Male: {:.1f}%'.format(gender_frames[1]['count'] / float(total) * 100, gender_frames[0]['count'] / float(total) * 100))
pd.DataFrame(list(gender_frames))

Female: 42.1%, Male: 57.9%


,count,name
0,569779,male
1,414019,female
2,512,unknown


In [3]:
# Get # of frames with a man vs. woman on them
shows = [t['show'] for t in Video.objects.distinct('show').values('show')]
counts = {}
for show in shows:
    print 'Computing for {}'.format(show)
    counts[show] = {}
    for gender in ['male', 'female']:
        counts[show][gender] = Frame.objects.filter(
            video__show=show, faceinstance__gender__name=gender).distinct('id').count()
    counts[show]['total'] = Frame.objects.filter(video__show=show).annotate(c=Count('faceinstance')).filter(c__gt=0).count()

print ''
    
for show, count in counts.iteritems():
    print('{}: female {:.1f}%, male {:.1f}%'.format(show, (count['female'] / float(count['total']) * 100), (count['male'] / float(count['total']) * 100)))
pd.DataFrame(counts)

Computing for Americas News Headquarters
Computing for CNN Newsroom
Computing for CNN Newsroom With Poppy Harlow
Computing for Fareed Zakaria GPS
Computing for On the Record With Brit Hume
Computing for Shepard Smith Reporting
Computing for Situation Room With Wolf Blitzer
Computing for Special Report With Bret Baier
Computing for The Five
Computing for The Lead With Jake Tapper
Computing for The Real Story With Gretchen Carlson
Computing for The Situation Room

Special Report With Bret Baier: female 31.9%, male 86.2%
The Real Story With Gretchen Carlson: female 61.0%, male 68.4%
On the Record With Brit Hume: female 42.8%, male 74.4%
Fareed Zakaria GPS: female 30.2%, male 78.2%
Shepard Smith Reporting: female 44.9%, male 75.5%
The Five: female 49.7%, male 61.6%
CNN Newsroom With Poppy Harlow: female 66.9%, male 69.5%
The Situation Room: female 47.2%, male 71.8%
Americas News Headquarters: female 47.8%, male 68.3%
Situation Room With Wolf Blitzer: female 47.6%, male 77.8%
The Lead With 

,Americas News Headquarters,CNN Newsroom,CNN Newsroom With Poppy Harlow,Fareed Zakaria GPS,On the Record With Brit Hume,Shepard Smith Reporting,Situation Room With Wolf Blitzer,Special Report With Bret Baier,The Five,The Lead With Jake Tapper,The Real Story With Gretchen Carlson,The Situation Room
female,21540,24188,32230,10786,20424,21052,28376,17113,24827,24010,27893,20404
male,30809,26844,33461,27944,35533,35379,46349,46213,30767,34600,31284,31040
total,45096,40831,48158,35725,47766,46878,59607,53591,49981,48407,45761,43212


In [20]:
# Get # of total faces per show that are man vs. woman
def talking_heads(qs):
    return qs.annotate(height=F('bbox_y2')-F('bbox_y1')).filter(height__gte=0.25)

shows = [t['show'] for t in Video.objects.distinct('show').values('show')]
counts = {}
for show in shows:
    counts[show] = {}
    for gender in ['male', 'female']:
        counts[show][gender] = talking_heads(FaceInstance.objects.filter(frame__video__show=show, gender__name=gender)).count()
    counts[show]['total'] = talking_heads(FaceInstance.objects.filter(frame__video__show=show)).count()    

print ''
    
for show, count in counts.iteritems():
    print('{}: female {:.1f}%, male {:.1f}%'.format(show, (count['female'] / float(count['total']) * 100), (count['male'] / float(count['total']) * 100)))
    
pd.DataFrame(counts)


Special Report With Bret Baier: female 11.9%, male 88.1%
The Real Story With Gretchen Carlson: female 44.1%, male 55.9%
On the Record With Brit Hume: female 27.8%, male 72.2%
Fareed Zakaria GPS: female 17.7%, male 82.3%
Shepard Smith Reporting: female 24.6%, male 75.4%
The Five: female 39.8%, male 60.2%
CNN Newsroom With Poppy Harlow: female 40.3%, male 59.7%
The Situation Room: female 31.7%, male 68.3%
Americas News Headquarters: female 34.8%, male 65.2%
Situation Room With Wolf Blitzer: female 29.5%, male 70.5%
The Lead With Jake Tapper: female 32.0%, male 68.0%
CNN Newsroom: female 41.0%, male 59.0%


,Americas News Headquarters,CNN Newsroom,CNN Newsroom With Poppy Harlow,Fareed Zakaria GPS,On the Record With Brit Hume,Shepard Smith Reporting,Situation Room With Wolf Blitzer,Special Report With Bret Baier,The Five,The Lead With Jake Tapper,The Real Story With Gretchen Carlson,The Situation Room
female,13314,10516,12973,4358,12173,8451,16786,6917,15401,12898,20558,9136
male,24925,15161,19222,20223,31549,25889,40166,51373,23336,27364,26099,19649
total,38239,25677,32195,24581,43722,34340,56969,58313,38737,40262,46657,28785


In [9]:
# Get # of total faces per show that are man vs. woman
def talking_heads(qs):
    return qs.annotate(height=F('bbox_y2')-F('bbox_y1')).filter(height__gte=0.25)

id = 4457280
FaceFeatures.dropTempFeatureModel()
FaceFeatures.getTempFeatureModel([id])

shows = ['CNN Newsroom With Poppy Harlow']
counts = {}
for show in shows:
    counts[show] = {}
    for gender in ['male', 'female']:
        counts[show][gender] = talking_heads(FaceInstance.objects.filter(frame__video__show=show, gender__name=gender, facefeaturestemp__distto_4457280__gte=1.7)).count()
    counts[show]['total'] = talking_heads(FaceInstance.objects.filter(frame__video__show=show, facefeaturestemp__distto_4457280__gte=1.7)).count()    

print ''
    
for show, count in counts.iteritems():
    print('{}: female {:.1f}%, male {:.1f}%'.format(show, (count['female'] / float(count['total']) * 100), (count['male'] / float(count['total']) * 100)))
    
pd.DataFrame(counts)


CNN Newsroom With Poppy Harlow: female 29.9%, male 70.1%


,CNN Newsroom With Poppy Harlow
female,2584
male,6062
total,8646
